In [1]:
# in order to install the required packages for the snn conversion, 
# you need the following packages
# furthemore you must have installed miniconda or anaconda
# and activated a virtual environment to execute the following commands

# %conda install akida
# %conda install cnn2snn
# %conda install akida-models

In [1]:
import tensorflow as tf
import utils

c:\Users\fmila\miniconda3\envs\studienarbeit\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_train, ds_train_info = utils.getDataset('train')
ds_test, ds_test_info = utils.getDataset('test')
ds_eval, ds_eval_info = utils.getDataset('validation')





 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 




In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

In [5]:
model_file = get_file(
    "akidanet_imagenet_224_alpha_50.h5",
    "http://data.brainchip.com/models/akidanet/akidanet_imagenet_224_alpha_50.h5",
    cache_subdir='models/akidanet_imagenet')

model = Sequential()
model.add(load_model(model_file)) 
model.add(Dense(4, activation='relu'))

5714072/5714072 [==============================] - 1s 0us/step


In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"), 
              metrics=['accuracy'])
model.get_layer(index=-1)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./imagenet_models",
                                                 save_weights_only=True,
                                                 verbose=1)

In [7]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
37/37 [==============================] - 195s 4s/step - loss: 0.3460 - accuracy: 0.2412
Epoch 2/6
37/37 [==============================] - 125s 3s/step - loss: 0.2430 - accuracy: 0.1311
Epoch 3/6
37/37 [==============================] - 127s 3s/step - loss: 0.1696 - accuracy: 0.0298
Epoch 4/6
37/37 [==============================] - 124s 3s/step - loss: 0.1210 - accuracy: 0.7009
Epoch 5/6
37/37 [==============================] - 122s 3s/step - loss: 0.0901 - accuracy: 0.7612
Epoch 6/6
37/37 [==============================] - 115s 3s/step - loss: 0.0703 - accuracy: 0.7612


In [8]:
results = model.evaluate(
  ds_eval
)

print("test loss, test acc:", results)

model.save("models/imagenet_model")

9/9 [==============================] - 11s 707ms/step - loss: 0.0632 - accuracy: 0.7594
test loss, test acc: [0.06319066882133484, 0.759358286857605]


INFO:tensorflow:Assets written to: models/imagenet_model\assets


INFO:tensorflow:Assets written to: models/imagenet_model\assets


In [3]:
import cv2
import numpy as np

model = tf.keras.models.load_model("models/imagenet_model")

In [5]:
for ele in ds_eval.take(1):

  predictions = tf.convert_to_tensor([np.array([bbox]) for bbox in model.predict(ele[0])])

  display(predictions)
  #display(ele[1])

  images = tf.image.draw_bounding_boxes(
    ele[0], ele[1], [(0, 0, 255) for _ in range(len(ele[0]))], name=None
    )
  
  images = tf.image.draw_bounding_boxes(
    images, predictions, [(0, 255, 0) for _ in range(len(images))], name=None
    )
  
  utils.display_imgs(images)  

4/4 [==============================] - 2s 168ms/step


<tf.Tensor: shape=(128, 1, 4), dtype=float32, numpy=
array([[[0.20320007, 0.39182246, 0.40835583, 0.4579454 ]],

       [[0.2032034 , 0.39182884, 0.4083622 , 0.45795247]],

       [[0.20320195, 0.39182606, 0.4083594 , 0.4579494 ]],

       [[0.20320371, 0.39182943, 0.40836278, 0.4579531 ]],

       [[0.20320323, 0.39182854, 0.4083619 , 0.4579521 ]],

       [[0.20320241, 0.391827  , 0.40836036, 0.45795038]],

       [[0.20320386, 0.39182973, 0.4083631 , 0.45795345]],

       [[0.20320246, 0.39182705, 0.40836042, 0.45795047]],

       [[0.20319985, 0.39182204, 0.40835539, 0.45794493]],

       [[0.20320237, 0.3918269 , 0.40836027, 0.4579503 ]],

       [[0.20319901, 0.3918204 , 0.40835378, 0.4579431 ]],

       [[0.2031992 , 0.39182076, 0.40835407, 0.4579435 ]],

       [[0.20320095, 0.39182416, 0.4083575 , 0.45794728]],

       [[0.20320193, 0.39182603, 0.4083594 , 0.45794937]],

       [[0.20320278, 0.39182767, 0.40836105, 0.4579512 ]],

       [[0.20320126, 0.39182472, 0.40835807, 0.

0
1
2
3
4
5
6


IndexError: index 6 is out of bounds for axis 0 with size 6

: 

: 

In [11]:

for box in [bbox, prediction]:

  x = box[0]
  y = box[1]
  xx = box[2]
  yy = box[3]

  print("x ", x, "y ", y, "xx ", xx, "yy", yy)

  cv2.rectangle(image, (x, y), (xx, yy), (0, 0, 255), 2)
  print("x,y,w,h:",x,y,xx,yy)
  
# save resulting image
cv2.imwrite('example.jpg', image)      

# show thresh and result    
cv2.imshow("bounding_box", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'bbox' is not defined